## Load data

In [1]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
random_state = 12041500

In [2]:
df_train = pd.read_json("./data/synthetic_data_CTGANSynthesizer.json")
df_test = pd.read_json("./data/testset.json")

df_train.drop(columns=['fnlwgt'], inplace=True)
df_test.drop(columns=['fnlwgt'], inplace=True)

ratio_features = ["age", "capital-gain", "capital-loss", "hours-per-week"]
ordinal_features = ["education-num"]
nominal_features = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex']
target = 'income'

## Train baseline

In [3]:
from sklearn.linear_model import LogisticRegression
from utils import create_model, train_and_evaluate, describe_model

In [4]:
clf = LogisticRegression(max_iter=1000, random_state=random_state)

In [5]:
## Train baseline
model = create_model(clf, nominal_features)
y_test, y_pred = train_and_evaluate(model, df_train, df_test, target, drop_na=True)
metrics = describe_model(y_test, y_pred, verbose=True)

Metric         Value               
Accuracy       0.8361032716630082
Precision      0.7822701965699763
Recall         0.7458442654631545
F1             0.7608356769787064


## Fairness Evaluation

In [6]:
from utils import split_data
from utils_fairness import search_bias, calc_fairness_score, explain_detected_bias

In [7]:
X_train, y_train = split_data(df_train, target, drop_na=True)

In [8]:
model = create_model(clf, nominal_features)
model.fit(X_train, y_train)
probs = pd.Series(model.predict_proba(X_train)[:, 1]) # we select for target label

In [9]:
privileged_subset, _ = search_bias(X_train, y_train, probs, 1, penalty=5)

In [10]:
print(privileged_subset)

({'capital-gain': [0, 1, 6]}, 210.8649)


In [11]:
calc_fairness_score(df_train, privileged_subset[0].keys(), target, verbose=True)

Sensitive Attributes: ['capital-gain']

                         Group Distance  Proportion  Counts   P-Value
capital-gain [37.00, 16383.00]    0.434    0.095449    3563  0.00e+00
    capital-gain [-0.00, 5.00]   -0.137    0.517078   19302 1.48e-323
   capital-gain [23.00, 37.00]    0.253    0.097458    3638 3.41e-256
   capital-gain [15.00, 23.00]    0.087    0.106352    3970  1.82e-38
    capital-gain [5.00, 10.00]   -0.047    0.097645    3645  2.44e-13

Weighted Mean Statistical Distance: 0.1648489023109496


In [12]:
explain_detected_bias(df_train, probs, target, privileged_subset[0])

Our detected privileged group has a size of 17732, we observe 0.0594 as the average probability of earning >50k, but our model predicts 0.2059


## Fairness Metrics

In [13]:
from utils_fairness import transform_to_bias_dataset, describe_fairness, scan_and_calculate_fairness, plot_fairness_metrics

In [14]:
df_train_bias = transform_to_bias_dataset(df_train, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)
df_test_bias = transform_to_bias_dataset(df_test, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)

1744 Na rows removed!
202 Na rows removed!


In [15]:
model = create_model(clf, nominal_features)
y_test, y_pred = train_and_evaluate(model, df_train, df_train, target, drop_na=True)
metrics = describe_fairness(df_train_bias[target], y_pred, list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   0.20866477995599425
average_odds_difference         0.18045635399467233
equal_opportunity_difference    0.2979438458132316
disparate_impact                6.0227746607021
theil_index                     0.11351788790247734


In [ ]:
df_fairness_metrics, priviliged_subsets = pd.DataFrame(
    columns=[
        "statistical_parity_difference",
        "average_abs_odds_difference",
        "equal_opportunity_difference",
        "disparate_impact",
        "theil_index",
    ]
), {}

for i in [1e-17, 1e-10, 0.001, 0.01, 0.1, 0, 1, 5, 10, 25, 50, 100]:
    metrics, priv = scan_and_calculate_fairness(model, df_train, target, i)    
    df_fairness_metrics.loc[f"{i}"] = metrics.values()
    priviliged_subsets[f"{i}"] = priv

In [ ]:
df_fairness_metrics

In [ ]:
df_fairness_metrics.to_json("./results/fairness_metrics_synthetic.json")

## Mitigation

In [17]:
from tqdm import tqdm
from sklearn import clone
from aif360.algorithms.preprocessing import Reweighing
from utils_fairness import create_aif360_standardDataset, compute_metrics, reweight_mitigation
from utils import plot_metrics

### Reweighting

In [18]:
# create (un)privileged groups
privileged_groups = [{key: 1 for key in list(privileged_subset[0].keys())}]
unprivileged_groups = [{key: 0 for key in list(privileged_subset[0].keys())}]

In [19]:
# convert dataset
train_dataset = create_aif360_standardDataset(df_train.dropna(), nominal_features, target, 1, list(privileged_subset[0].keys()), list(privileged_subset[0].values()))
test_dataset = create_aif360_standardDataset(df_test.dropna(), nominal_features, target, 1, list(privileged_subset[0].keys()), list(privileged_subset[0].values()))

In [20]:
RW = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
RW.fit(train_dataset)

train_dataset_reweight = RW.transform(train_dataset)

#### Baseline

In [22]:
model = clone(clf)
model.fit(train_dataset.features, train_dataset.labels.ravel())
y_pred = model.predict(test_dataset.features)

_ = describe_model(test_dataset.labels.ravel(), y_pred, verbose=True)

Metric         Value               
Accuracy       0.8250235183443085
Precision      0.8011300969157298
Recall         0.6760386016778689
F1             0.7049576572819011


In [23]:
y_pred = model.predict(train_dataset.features)
_ = describe_fairness(df_train_bias[target], y_pred, list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   0.27325293449977367
average_odds_difference         0.3160490306583643
equal_opportunity_difference    0.5220668823049727
disparate_impact                20.055737563922488
theil_index                     0.10899556247160261


#### Mitigated Model

In [25]:
# mitigated model
model = clone(clf)
model.fit(train_dataset_reweight.features, train_dataset_reweight.labels.ravel(), sample_weight=train_dataset_reweight.instance_weights)
y_pred = model.predict(test_dataset.features)

_ = describe_model(test_dataset.labels.ravel(), y_pred, verbose=True)

Metric         Value               
Accuracy       0.8145709208738372
Precision      0.7467964552505164
Recall         0.7586925908420961
F1             0.752294520368006


In [26]:
y_pred = model.predict(train_dataset.features)
_ = describe_fairness(df_train_bias[target], y_pred, list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   0.10227744846927242
average_odds_difference         -0.005485384858981368
equal_opportunity_difference    -0.005484955838945593
disparate_impact                2.198612477012649
theil_index                     0.1333673780742846


#### Bias detection & mitigation until bias free

In [27]:
df_fairness_metrics = pd.DataFrame(
    columns=[
        "statistical_parity_difference",
        "average_abs_odds_difference",
        "equal_opportunity_difference",
        "disparate_impact",
        "theil_index",
    ]
)

In [ ]:
df_metrics = pd.DataFrame(
    columns=['acc', 'prec', 'rec', 'f1']
)

In [ ]:
weights = None

weights_hist = [weights]
max_iter = 10
for i in tqdm(range(max_iter)):
    X_train, y_train = split_data(df_train, target, True)
    X_test, y_test = split_data(df_test, target, True)
    weights, model_metrics, fair_metrics = reweight_mitigation(clf, nominal_features, target, X_train, y_train, X_test, y_test, penalty=1, sample_weights=weights)
    if model_metrics is None and fair_metrics is None and weights is None:
        break

    df_metrics.loc[f"mitigation_{i}"] = model_metrics.values()
    df_fairness_metrics.loc[f"mitigation_{i}"] = fair_metrics.values()
    weights_hist.append(weights)

### Fair Learning

In [28]:
from utils import prepare_data_fair_learning, plot_fairlearning_results
from utils_fairness import get_fair_learning_scoring
from sklearn.model_selection import GridSearchCV
from aif360.sklearn.preprocessing import LearnedFairRepresentations

In [29]:
df_train_bias = transform_to_bias_dataset(df_train, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)
df_test_bias = transform_to_bias_dataset(df_test, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)

X_train, y_train, X_test, y_test = prepare_data_fair_learning(df_train_bias, df_test_bias, nominal_features, target)

1744 Na rows removed!
202 Na rows removed!


In [30]:
max_delta = get_fair_learning_scoring(list(privileged_subset[0].keys()))

In [ ]:
lfr = LearnedFairRepresentations(
    list(privileged_subset[0].keys()),
    n_prototypes=25,
    max_iter=100,
    random_state=random_state,
)

In [ ]:
params = {
    "reconstruct_weight": [1e-2, 1e-3, 1e-4],
    "target_weight": [100, 1000],
    "fairness_weight": [0, 100, 1000],
}

In [ ]:
grid = GridSearchCV(lfr, params, scoring=max_delta, cv=3, n_jobs=-1).fit(
    X_train, y_train, priv_group=(1,) * len(list(privileged_subset[0].keys()))
)
res = pd.DataFrame(grid.cv_results_)

#### Baseline

In [33]:
model = clone(clf)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

_ = describe_model(y_test, y_pred, verbose=True)

Metric         Value               
Accuracy       0.8368349534859413
Precision      0.7828137452198869
Recall         0.7485532086541198
F1             0.7628348756731729


In [34]:
y_pred = model.predict(X_train)
_ = describe_fairness(df_train_bias[target], y_pred, list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   0.20656582123293724
average_odds_difference         0.1795749894343781
equal_opportunity_difference    0.2977239135212729
disparate_impact                6.014991986552083
theil_index                     0.11421882144989567


#### Using Grid itself

In [36]:
_ = describe_model(y_test, grid.predict(X_test), verbose=True)

Metric         Value               
Accuracy       0.7742238946378175
Precision      0.6846828630086876
Recall         0.5927048197548834
F1             0.6024177943160193


In [37]:
_ = describe_fairness(y_train, grid.predict(X_train), list(privileged_subset[0].keys()) ,verbose=True)

Metric                          Value               
statistical_parity_difference   0.2776604056164877
average_odds_difference         0.3444513447318348
equal_opportunity_difference    0.5376897044397566
disparate_impact                277.7784255045295
theil_index                     0.13564518741647613


#### Transforming data

In [38]:
model = clone(clf)
model.fit(grid.transform(X_train), y_train)
y_pred = model.predict(X_test)

_ = describe_model(y_test, y_pred, verbose=True)

Metric         Value               
Accuracy       0.7765234660813212
Precision      0.697120770494787
Recall         0.5827855156502163
F1             0.588704481752163


In [39]:
_ = describe_fairness(y_train, model.predict(X_train), list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   0.09807192268066527
average_odds_difference         0.11742834217467112
equal_opportunity_difference    0.1723249469536223
disparate_impact                            0.0
theil_index                     0.2012487441489287


##### Also Transforming test data

In [40]:
_ = describe_model(y_test, model.predict(grid.transform(X_test)), verbose=True)

Metric         Value               
Accuracy       0.7754782063342741
Precision      0.6889270638844567
Recall         0.5918972686632136
F1             0.6013375712648379


In [41]:
_ = describe_fairness(y_train, model.predict(grid.transform(X_train)), list(privileged_subset[0].keys()) ,verbose=True)

Metric                          Value               
statistical_parity_difference   0.2961085343142529
average_odds_difference         0.36685438686235855
equal_opportunity_difference    0.5702411508430346
disparate_impact                717.8364603556184
theil_index                     0.13084895074813588


Multiple Runs cannot be done, because of the dataset already needs to be one hot encoded for the mitigation, making the `search_bias` function unnecessary and returning not viable resolutions.

In [42]:
res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_fairness_weight,param_reconstruct_weight,param_target_weight,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,24.752142,1.777984,0.112692,0.072171,0,0.0100,100,"{'fairness_weight': 0, 'reconstruct_weight': 0...",-0.703722,-0.744387,-0.764539,-0.737549,0.025295,14
1,19.338813,0.354392,0.100726,0.010804,0,0.0100,1000,"{'fairness_weight': 0, 'reconstruct_weight': 0...",-0.569532,-0.609017,-0.432301,-0.536950,0.075733,1
2,14.814898,5.863008,0.096248,0.014033,0,0.0010,100,"{'fairness_weight': 0, 'reconstruct_weight': 0...",-0.565238,-0.620270,-0.525844,-0.570451,0.038725,7
3,18.801443,1.672079,0.107203,0.017980,0,0.0010,1000,"{'fairness_weight': 0, 'reconstruct_weight': 0...",-0.568084,-0.574992,-0.542222,-0.561766,0.014104,6
4,19.644145,0.481360,0.071161,0.021377,0,0.0001,100,"{'fairness_weight': 0, 'reconstruct_weight': 0...",-0.556896,-0.566217,-0.548289,-0.557134,0.007321,5
5,19.326477,0.717361,0.102469,0.077344,0,0.0001,1000,"{'fairness_weight': 0, 'reconstruct_weight': 0...",-0.588357,-0.496911,-0.560892,-0.548720,0.038312,3
6,21.813466,1.790828,0.082802,0.021522,100,0.0100,100,"{'fairness_weight': 100, 'reconstruct_weight':...",-0.763504,-0.762012,-0.706881,-0.744132,0.026348,15
7,16.207369,0.919578,0.135667,0.058018,100,0.0100,1000,"{'fairness_weight': 100, 'reconstruct_weight':...",-0.689450,-0.601783,-0.476128,-0.589120,0.087548,12
8,17.000117,0.234587,0.048463,0.008176,100,0.0010,100,"{'fairness_weight': 100, 'reconstruct_weight':...",-0.585105,-0.568125,-0.570999,-0.574743,0.007421,9
9,15.885579,1.262007,0.052776,0.018838,100,0.0010,1000,"{'fairness_weight': 100, 'reconstruct_weight':...",-0.594535,-0.587861,-0.481526,-0.554640,0.051772,4


### Fair Learning

In [43]:
%matplotlib inline
# Load all necessary packages

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult
from aif360.algorithms.preprocessing.lfr import LFR

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import numpy as np
from utils_fairness import create_aif360_standardDataset

In [44]:
# create (un)privileged groups
privileged_groups = [{key: 1 for key in list(privileged_subset[0].keys())}]
unprivileged_groups = [{key: 0 for key in list(privileged_subset[0].keys())}]

In [45]:
grid.best_params_

{'fairness_weight': 0, 'reconstruct_weight': 0.01, 'target_weight': 1000}

In [46]:
TR = LFR(unprivileged_groups=unprivileged_groups,
         privileged_groups=privileged_groups,
         k=5, Ax=0.01, Ay=1000, Az=0,
         verbose=1,
         seed=random_state
)

TR = TR.fit(train_dataset, maxiter=5000, maxfun=1000)

step: 0, loss: 693.9774324818318, L_x: 4050.095772408544,  L_y: 0.6534764747577464,  L_z: 0.0014173912912793184
step: 250, loss: 693.9774319689096, L_x: 4050.0957723490305,  L_y: 0.6534764742454193,  L_z: 0.0014173919018108771
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          250     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93977D+02    |proj g|=  2.98246D+01
step: 500, loss: 640.1850574596295, L_x: 4049.176292285174,  L_y: 0.5996932945367778,  L_z: 0.0009651204130496499

At iterate    1    f=  6.40185D+02    |proj g|=  2.11041D+01
step: 750, loss: 608.5644683903249, L_x: 4042.221310598115,  L_y: 0.5681422552843437,  L_z: 0.001024285487798589
step: 1000, loss: 577.2502664616709, L_x: 4047.420010606822,  L_y: 0.5367760663556027,  L_z: 0.0007042036313121586

At iterate    2    f=  5.77250D+02    |proj g|=  1.78881D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of 

In [50]:
# Transform training data and align features
train_dataset_lfr = TR.transform(train_dataset)

In [51]:
cnt = {}
for i in dataset_transf_train.labels.ravel():
    cnt[i] = cnt.get(i, 0) + 1

cnt

{0.0: 37329}

#### Baseline

In [56]:
model = clone(clf)
model.fit(train_dataset.features, train_dataset.labels.ravel())
y_pred = model.predict(test_dataset.features)

_ = describe_model(test_dataset.labels.ravel(), y_pred, verbose=True)

Metric         Value               
Accuracy       0.8250235183443085
Precision      0.8011300969157298
Recall         0.6760386016778689
F1             0.7049576572819011


In [57]:
y_pred = model.predict(train_dataset.features)
_ = describe_fairness(df_train_bias[target], y_pred, list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   0.27325293449977367
average_odds_difference         0.3160490306583643
equal_opportunity_difference    0.5220668823049727
disparate_impact                20.055737563922488
theil_index                     0.10899556247160261


#### Mitigation

In [55]:
_ = describe_model(df_test_bias[target], [0]*len(df_test_bias[target]), verbose=True)

Metric         Value               
Accuracy       0.7594857322044528
Precision      0.3797428661022264
Recall                     0.5
F1             0.4316521119230084


In [53]:
_ = describe_fairness(df_train_bias[target], np.array([0]*len(df_train_bias)), list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference               0.0
average_odds_difference                     0.0
equal_opportunity_difference                0.0
disparate_impact                            0.0
theil_index                     0.22869080098773423


### Disparate Impact Remover

In [58]:
import numpy as np
from tqdm import tqdm

from aif360.algorithms.preprocessing import DisparateImpactRemover

In [59]:
index = train_dataset.feature_names.index('capital-gain')

In [60]:
fairness_metrics = []
utility_metrics = []
for level in tqdm(np.linspace(0, 1, 10)):
    di = DisparateImpactRemover(repair_level=level)
    train_repd = di.fit_transform(train_dataset)
    test_repd = di.fit_transform(test_dataset)
    
    X_tr = np.delete(train_repd.features, index, axis=1)
    X_te = np.delete(test_repd.features, index, axis=1)
    y_tr = train_repd.labels.ravel()
    
    model = clone(clf)
    model.fit(X_tr, y_tr)

    utility_metrics.append(describe_model(test_repd.labels.ravel(), model.predict(X_te), verbose=False))
    fairness_metrics.append(describe_fairness(df_train_bias[target], model.predict(X_tr), list(privileged_subset[0].keys()), verbose=False))


  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:49<00:00,  9.81s/it]
